In [42]:
import findspark

import pandas
import plotly as p
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.figure_factory as ff
import plotly.plotly as py
import plotly.graph_objs as go
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from plotly.widgets import GraphWidget

findspark.init("C:\spark-2.2.1-bin-hadoop2.7")
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
sparkSession = SparkSession.builder.appName(
    "option-pricer-write-to-hadoop").getOrCreate()

<IPython.core.display.Javascript object>

In [21]:


p.tools.set_credentials_file(username='lalor188', api_key='MQxJd3Rs9X5aKOZiDEgt')

resultsHiveDF = sparkSession.read.format('json').load('E:\ProjectDB')

resultsHiveDF.createOrReplaceTempView("optionData")

tickers = sparkSession.sql("SELECT Ticker FROM optionData").collect()
tickerSymbols = []
for i in tickers: # remove Row(Ticker=) from ticker
   tickerSymbols.append(i['Ticker'])

expDate = sparkSession.sql("SELECT ExpirationDate FROM optionData WHERE Ticker Like 'MMM'").collect()

riskFreeRates = sparkSession.sql("SELECT riskFreeRates FROM optionData WHERE Ticker Like 'MMM'").collect()

strikePrices = sparkSession.sql("SELECT Prices['StrikePrice'] FROM optionData WHERE Ticker Like 'ZTS'").collect()

numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like 'ZTS'").collect()



In [3]:
# Cell to declare all funcions

# Function to get prices based on a specific ticker symbol
def getOptionPricesByTicker(symbol):
    return sparkSession.sql("SELECT Prices FROM optionData WHERE Ticker Like '" + symbol + "'").collect()

# Function to turn int to array of ints for number of days before expDate
# eg. number of says is 5 returns [1,2,3,4,5]
def getNumberDaysArray(x):
    numDaysArr = []
    for i in range(1, x[0]['NumberOfDays'] +1):
        numDaysArr.append(i)
    return numDaysArr

def getPricesByStrike(optionPrices, strike):
    allPrices = []
    for i in optionPrices[0][0]:
        if i != None and i['StrikePrice'] == strike:
            allPrices.append(i[0])
    return allPrices

def getPricesByRiskFreeRate(optionPrices, RiskFreeRate):
    allPrices = []
    for i in optionPrices[0][0]:
        if i != None and i['RiskFreeRate'] == RiskFreeRate:
            allPrices.append(i[0])
    return allPrices


def getPricesByStrikeAndRate(optionPrices, strike, RiskFreeRate):
    allPrices = []
    for i in optionPrices[0][0]:
        if i != None and i['StrikePrice'] == strike and i['RiskFreeRate'] == RiskFreeRate:
            allPrices.append(i[0])
    return allPrices

def getCalls(optionPrices):
    callPrices = []
    for i in optionPrices[0][0]:
        if i['Call'] != None:
            callPrices.append(i['Call'])
    return callPrices

def getPuts(optionPrices):
    putPrices = []
    for i in optionPrices[0][0]:
        if i['Put'] != None:
            putPrices.append(i['Put'])
    return putPrices

def priceToArray(opPrices):
    prices = {}
    count = 1
    for i in opPrices:
        priceArr = []
        for x in i:
            if x != None:
                priceArr.append(x)
        prices['opt'+str(count)] = (priceArr.copy())
        count = count + 1
    return prices

In [4]:
prices = getOptionPricesByTicker('ZTS')

callPrices = []
putPrices = []

callPricesStrike = getPricesByStrike(prices, 85.0)

callPricesRate = getPricesByRiskFreeRate(prices, 1.67)

callPricesSR = getPricesByStrikeAndRate(prices, 67.5, 1.67)

callPrices = getCalls(prices)

putPrices = getPuts(prices)
        
numberOfDays = getNumberDaysArray(numOfDays)
    

In [92]:
optionTicker = widgets.Dropdown(
    options=tickerSymbols,
    value='MMM',
    description='Option Ticker symbol:',
)
g = GraphWidget('https://plot.ly/~kevintest/1192/')

In [93]:

        
def response2(change):
    # print "using date"
    if optionTicker.value:
        filter_list = optionTicker.value
        data = []
        opPriceDic = priceToArray(getCalls(getOptionPricesByTicker(filter_list)))
        x = numberOfDays
        g.restyle({'x': [x], 'type': 'line', 
                   'opacity': 0.75}, indices=0) 
        for key, values in opPriceDic.items():
            g.restyle({'x': [values], 'name': 'Departure Delays', 'type': 'line', 
                   'opacity': 0.75}, indices=1)
            g.relayout({'barmode': 'overlay', 'xaxis': {'title': 'Days to expiration'},
                'yaxis': {'title': 'Price ($)'}})
    



In [94]:

optionTicker.observe(response2, names="value")
container2 = widgets.HBox([optionTicker])
display(container2)
display(g)

HBox(children=(Dropdown(description='Option Ticker symbol:', index=99, options=('AMZN', 'GOOGL', 'PCLN', 'GOOG', 'ROK', 'MCHP', 'AZO', 'SPG', 'NFLX', 'FDX', 'NTRS', 'Q', 'CMG', 'BA', 'AAPL', 'NVDA', 'STX', 'MU', 'AGN', 'CELG', 'CRM', 'BIIB', 'ADSK', 'GILD', 'TGT', 'WMT', 'CB', 'ISRG', 'MSFT', 'CHTR', 'C', 'LOW', 'NTAP', 'LMT', 'BCR', 'JPM', 'AMAT', 'CAT', 'DAL', 'VLO', 'GM', 'VMC', 'BBY', 'LRCX', 'PYPL', 'REGN', 'FB', 'ULTA', 'INTC', 'XOM', 'QCOM', 'UAL', 'WDC', 'GS', 'BLK', 'MO', 'WYNN', 'ILMN', 'AVGO', 'V', 'ABC', 'SCHW', 'KSS', 'UNP', 'ORCL', 'ABBV', 'RHT', 'ATVI', 'SLB', 'JNJ', 'ALGN', 'MGM', 'DE', 'PG', 'COF', 'NEM', 'AXP', 'NKE', 'CVX', 'CVS', 'WFC', 'SHW', 'HD', 'ESRX', 'MCD', 'DG', 'ADI', 'NOC', 'DLTR', 'CBOE', 'MNST', 'TXN', 'UPS', 'EA', 'ADBE', 'GE', 'URI', 'COST', 'VZ', 'MMM', 'CSCO', 'RCL', 'UNH', 'MHK', 'DWDP', 'NSC', 'HCN', 'HCA', 'CMCSA', 'GPS', 'DIS', 'UTX', 'LUV', 'F', 'EXPE', 'CLX', 'DISH', 'IBM', 'APC', 'GPN', 'AIG', 'NWL', 'CBS', 'AAL', 'MDLZ', 'HAL', 'KO', 'BMY', 'PXD', 'APA', 'AMGN', 'HON', 'SWKS', 'BAC', 'XLNX', 'SYF', 'ANTM', 'TRIP', 'CAH', 'GWW', 'MOS', 'M', 'DVN', 'PM', 'T', 'PEP', 'EFX', 'EBAY', 'MRK', 'MAR', 'WBA', 'GRMN', 'HUM', 'NUE', 'AKAM', 'TSCO', 'AMT', 'EOG', 'CMI', 'KSU', 'SBUX', 'HAS', 'ANDV', 'LYB', 'PFE', 'INCY', 'GD', 'CME', 'IP', 'HES', 'FCX', 'EQIX', 'MDT', 'OXY', 'KR', 'KORS', 'BDX', 'TIF', 'MS', 'ABT', 'AMD', 'WHR', 'DFS', 'PSX', 'CTL', 'COP', 'CCL', 'MET', 'GLW', 'CSX', 'MPC', 'ALXN', 'ITW', 'HOG', 'DOV', 'DHI', 'PCG', 'NRG', 'ETFC', 'VFC', 'ADM', 'EMR', 'MA', 'FFIV', 'LLY', 'KMI', 'USB', 'TWX', 'ETN', 'RRC', 'CF', 'TROW', 'TAP', 'YUM', 'MYL', 'RL', 'FL', 'HSY', 'VIAB', 'STZ', 'CI', 'KMX', 'NOV', 'MON', 'VRTX', 'TMO', 'CTSH', 'AFL', 'MCK', 'NFX', 'TXT', 'CL', 'BK', 'ZTS', 'EW', 'TSN', 'OKE', 'CXO', 'HOLX', 'CNC', 'GPC', 'HP', 'DHR', 'PPG', 'SWK', 'PSA', 'KMB', 'ED', 'ADP', 'PH', 'AAP', 'MRO', 'BBT', 'PDCO', 'XL', 'BAX', 'ALB', 'GT', 'CBG', 'HIG', 'FLR', 'HRL', 'SYK', 'ADS', 'KLAC', 'AMG', 'PRU', 'WMB', 'MAT', 'UAA', 'BHGE', 'PX', 'PVH', 'IR', 'PHM', 'AMP', 'ACN', 'BRK.B', 'SNA', 'ECL', 'APD', 'SBAC', 'HPQ', 'TPR', 'NCLH', 'BSX', 'CTXS', 'QRVO', 'ORLY', 'EMN', 'CAG', 'NDAQ', 'KEY', 'ARNC', 'SO', 'JNPR', 'RTN', 'MTD', 'HPE', 'AET', 'ICE', 'PAYX', 'CERN', 'DLR', 'EQT', 'VRSN', 'FISV', 'SYMC', 'XEC', 'COTY', 'CPB', 'K', 'JCI', 'PRGO', 'CDNS', 'SPGI', 'INTU', 'MLM', 'AYI', 'AIZ', 'TDG', 'KHC', 'COG', 'WY', 'NLSN', 'EVHC', 'UA', 'ZBH', 'NEE', 'ETR', 'CINF', 'FOXA', 'ALK', 'SEE', 'SNI', 'PNC', 'TJX', 'BXP', 'IDXX', 'PWR', 'JWN', 'HLT', 'HRB', 'O', 'FMC', 'FITB', 'CHRW', 'LB', 'FE', 'DXC', 'IT', 'SYY', 'ROST', 'PPL', 'LH', 'MAS', 'ALL', 'UNM', 'GGP', 'IVZ', 'LEN', 'EXC', 'ANSS', 'LUK', 'IPG', 'MCO', 'WYN', 'MAC', 'GIS', 'PCAR', 'UHS', 'VRSK', 'EL', 'MKC', 'STT', 'TRV', 'HBI', 'AWK', 'VAR', 'DPS', 'CMA', 'XRX', 'FLIR', 'RF', 'VTR', 'OMC', 'RJF', 'MSI', 'VNO', 'FTI', 'EXR', 'SCG', 'CHK', 'LNC', 'HRS', 'SNPS', 'WAT', 'D', 'DRI', 'DUK', 'JBHT', 'WM', 'LKQ', 'DISCK', 'AVB', 'SRCL', 'IRM', 'HST', 'CCI', 'LLL', 'BWA', 'CTAS', 'ROP', 'FBHS', 'MAA', 'JEC', 'AME', 'COO', 'FRT', 'FOX', 'SJM', 'A', 'CA', 'RHI', 'MTB', 'PKG', 'DVA', 'EIX', 'SIG', 'ALLE', 'HSIC', 'PKI', 'AEP', 'SRE', 'RMD', 'WLTW', 'FTV', 'COL', 'EQR', 'FAST', 'AON', 'STI', 'NBL', 'WU', 'IFF', 'DGX', 'FIS', 'TEL', 'PGR', 'WRK', 'AVY', 'BHF', 'PEG', 'HBAN', 'RE', 'CNP', 'PNR', 'XYL', 'PFG', 'AEE', 'FLS', 'TSS', 'BEN', 'CSRA', 'DISCA', 'L', 'ESS', 'ARE', 'NI', 'EXPD', 'AES', 'BLL', 'ZION', 'DLPH', 'RSG', 'XEL', 'LEG', 'APH', 'CMS', 'BF.B', 'SLG', 'XRAY', 'HCP', 'PBCT', 'ES', 'WEC', 'CFG', 'KIM', 'AIV', 'CHD', 'AOS', 'PLD', 'LNT', 'DTE', 'MMC', 'TMK', 'NAVI', 'PNW', 'AJG', 'DRE', 'REG', 'UDR', 'NWSA', 'INFO', 'NWS'), value='MMM'),))

GraphWidget()

In [96]:
# Code for calls by ticker
data = []
opPriceDic = priceToArray(getCalls(getOptionPricesByTicker('ZTS')))
for key, values in opPriceDic.items():
    trace = go.Scatter(
        x = numberOfDays,
        y = values,
        name = 'option',
        line = dict(
            width = 4)
    )
    data.append(trace)
# Edit the layout
layout = dict(title = 'Call prices for ' ,
              xaxis = dict(title = 'Days to Expiration'),
              yaxis = dict(title = 'Price '),
              )
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='calls For ')


In [6]:
# Code for put by ticker
data = []
opPriceDic = priceToArray(getPuts(getOptionPricesByTicker('ZTS')))
for key, values in opPriceDic.items():
    trace = go.Scatter(
        x = numberOfDays,
        y = values,
        name = 'option',
        line = dict(
            width = 4)
    )
    data.append(trace)
# Edit the layout
layout = dict(title = 'Call prices for ' ,
              xaxis = dict(title = 'Days to Expiration'),
              yaxis = dict(title = 'Price '),
              )
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='calls For ')

In [84]:
import plotly.plotly as py
import plotly.graph_objs as go 



data = []
opPriceDic = priceToArray(getPuts(getOptionPricesByTicker('ZTS')))
for key, values in opPriceDic.items():
    trace = go.Scatter(
        x = numberOfDays,
        y = values,
        name = 'option',
        line = dict(
            width = 4)
    )
    data.append(trace)

high_annotations=[dict(x='2016-03-01',
                       y=5,
                       xref='x', yref='y',
                       text='High Average:<br>'+str(6),
                       ax=0, ay=-40),
                  dict(x=5,
                       y=5,
                       xref='x', yref='y',
                       text='High Max:<br>'+str(6),
                       ax=0, ay=-40)]
low_annotations=[dict(x='2015-05-01',
                      y=1,
                      xref='x', yref='y',
                      text='Low Average:<br>'+str(1),
                      ax=0, ay=40),
                 dict(x=1,
                      y=1,
                      xref='x', yref='y',
                      text='Low Min:<br>'+str(1),
                      ax=0, ay=40)]

updatemenus = list([
    dict(active=-1,
         buttons=list([   
            dict(label = 'MMM',
                 method = 'update',
                 args = [{'visible': [True, True, False, False]},
                         {'title': 'Yahoo High',
                          'annotations': high_annotations}]),
            dict(label = 'Low',
                 method = 'update',
                 args = [{'visible': [False, False, True, True]},
                         {'title': 'Yahoo Low',
                          'annotations': low_annotations}]),
            dict(label = 'Both',
                 method = 'update',
                 args = [{'visible': [True, True, True, True]},
                         {'title': 'Yahoo',
                          'annotations': high_annotations+low_annotations}]),
            dict(label = 'Reset',
                 method = 'update',
                 args = [{'visible': [True, False, True, False]},
                         {'title': 'Yahoo',
                          'annotations': []}])
        ]),
    )
])

layout = dict(title='Yahoo', showlegend=False,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='update_dropdown')

In [87]:
import plotly.plotly as py
from plotly.graph_objs import *

import numpy as np

s = np.linspace(0, 2 * np.pi, 240)
t = np.linspace(0, np.pi, 240)
tGrid, sGrid = np.meshgrid(s, t)

r = 2 + np.sin(7 * sGrid + 5 * tGrid)  # r = 2 + sin(7s+5t)
x = r * np.cos(sGrid) * np.sin(tGrid)  # x = r*cos(s)*sin(t)
y = r * np.sin(sGrid) * np.sin(tGrid)  # y = r*sin(s)*sin(t)
z = r * np.cos(tGrid)                  # z = r*cos(t)

surface = Surface(x=x, y=y, z=z)
data = Data([surface])

layout = Layout(
    title='Parametric Plot',
    scene=Scene(
        xaxis=XAxis(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=YAxis(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=ZAxis(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    )
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='parametric_plot')